In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import warnings 
warnings =filterwarnings('ignore')

In [2]:
df = pd.read_csv(r'C:\Users\Vinay\Downloads\creeplifesuperalloy (1).csv')

In [4]:
df1=df.drop(columns="Generation")

In [5]:
from sklearn.preprocessing import StandardScaler

In [6]:
scaler = StandardScaler()
scaler.fit(df1)
scaled_df1 = scaler.transform(df1)

In [ ]:
cmap = sn.cubehelix_palette(as_cmap=True,rot=-.3,light=1)
sn.clustermap(scaled_df1,cmap=cmap,linewidths=.2,figsize=(8,8));

In [8]:
from sklearn.cluster import KMeans
clusters=KMeans(5,random_state=42)
clusters.fit(scaled_df1)
df1["clusterid"] = clusters.labels_

In [9]:
df3= df1[df1.clusterid ==1]
df4= df1[df1.clusterid ==2]
df5= df1[df1.clusterid ==3]
df6= df1[df1.clusterid ==4]
df2= df1[df1.clusterid ==0]

In [ ]:
plt.figure(figsize=(8,6))
sn.countplot(x='clusterid',hue='clusterid',data=df1)

In [11]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
pca.fit(scaled_df1)
pca_df1 = pca.transform(scaled_df1)

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(pca_df1[:, 0],pca_df1[:, 1],c=df1['clusterid'],marker=None,cmap='plasma')
plt.xlabel('First principle component')
plt.ylabel('Second principle component')

In [114]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df5[["Constrained_lattice_misfit","temperature","applied_Stress","Ni_wt","Re_wt","Co_wt","Al_wt","Ti_wt","W_wt","Mo_wt","Cr_wt","Ta_wt","C_wt","Hf_wt","Ru_wt",
                                                      "B_wt","Y_wt","solution_treatment_time","1st_step_aging_treatment_time","2nd_step_aging_treatment_time","1st_step_aging_treatment_temperature","2nd_step_aging_treatment_temperature",
                                                      "τ","solution_treatment_temperature","DL","G","L","Ni3Al_fraction"]],df5["creep_life"],test_size=0.1,random_state=42)

In [27]:
scaler = StandardScaler()
scaler.fit(df3)
scaled_df1 = scaler.transform(df3)

In [ ]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train,y_train)

In [ ]:
from sklearn.linear_model import Ridge
ridge= Ridge(alpha =2,max_iter =500)
ridge.fit(X_train,y_train)

In [37]:
from sklearn.linear_model import Lasso
lasso=Lasso(alpha = 0.01,max_iter =10000)
lasso.fit(X_train,y_train)

C:\Users\Vinay\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 360.24635272558953, tolerance: 2.449734774193549
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.01, max_iter=10000)

In [86]:

    from sklearn.svm import SVR
    parameter = ' svr_model '
    svm = SVR(kernel='linear', C=1400, gamma='auto')
    

In [ ]:
from sklearn.ensemble import RandomForestClassifier
radm_clf=RandomForestClassifier(max_depth=1,n_estimators=10)
radm_clf.fit(X_train,y_train)

In [85]:
linreg_pred=linreg.fit(X_train,y_train,sample_weight=None )

In [118]:
y_pred=linreg.predict(X_train)

In [ ]:
test_pred_df = pd.DataFrame({'actual':y_train,'predicted':np.round(y_pred,2),'residuals':y_train-y_pred})
test_pred_df.sample()

In [ ]:
test_pred_df.describe()

In [78]:
def fit_poly(degree):
    p = np.polyfit(test_pred_df.actual,test_pred_df.predicted,deg = degree)
    test_pred_df['fit']=np.polyval(p,test_pred_df.actual)
    sn.regplot(test_pred_df.actual,test_pred_df.predicted,fit_reg=False)
    return plt.plot(test_pred_df.actual,test_pred_df.predicted,label='fit')

In [ ]:

plt.scatter(test_pred_df.actual,test_pred_df.predicted);
plt.xlabel("actual values")
plt.ylabel("predicted values")

In [ ]:

    from sklearn.gaussian_process import GaussianProcessRegressor
    from sklearn.gaussian_process.kernels import RationalQuadratic, WhiteKernel, RBF
    from sklearn.gaussian_process.kernels import ConstantKernel as C
    # parameter = "C(1, (1e-4, 10000)) * RationalQuadratic(alpha=0.01, length_scale_bounds=(1e-5, 20))"
    parameter = ' gaussian_model '
    # kernel_5 = C(1, (1e-4, 1)) * RationalQuadratic(alpha=0.1, length_scale_bounds=(0.01, 2000))
    kernel = C(1, (0.1, 10)) * RationalQuadratic(alpha=0.01, length_scale_bounds=(0.1, 1500))
    gpr = GaussianProcessRegressor(kernel=kernel, alpha=0.008, n_restarts_optimizer=20)
    gpr.fit(X_train,y_train)

In [ ]:
from sklearn import metrics
r2 = metrics.r2_score(y_train,linreg.predict(X_train))
print("R Squared:",r2)

In [ ]:
def get_train_test_rmse(model):
    y_train_pred = model.predict(X_train)
    rmse_train = round(np.sqrt(metrics.mean_squared_error(y_train,y_train_pred)),2)
    y_test_pred = model.predict(X_test)
    rmse_test =round(np.sqrt(metrics.mean_squared_error(y_test,y_test_pred)),2)
    print("train:",rmse_train,"test:",rmse_test)

In [188]:
get_train_test_rmse(gpr)

train: 0.95 test: 495.05


In [ ]:
linreg.coef_

In [ ]:
svm.coef_

In [34]:
columns_coef_df = pd.DataFrame({'columns':df2[["Constrained_lattice_misfit","temperature","applied_Stress","Ni_wt","Re_wt","Co_wt","Al_wt","Ti_wt","W_wt","Mo_wt","Cr_wt","Ta_wt","C_wt","Hf_wt","Ru_wt",
                                                      "B_wt","Y_wt","solution_treatment_time","1st_step_aging_treatment_time","2nd_step_aging_treatment_time","1st_step_aging_treatment_temperature","2nd_step_aging_treatment_temperature",
                                                      "τ","solution_treatment_temperature","DL","G","L","Ni3Al_fraction"]],'coef':linreg.coef_ })
sorted_coef_vals = columns_coef_df.sort_values('coef',ascending=False)

In [ ]:
plt.figure(figsize = (8,6))
sn.barplot(x="coef",y="columns",data=sorted_coef_vals);
plt.xlabel("Coefficients of Linear Regression")
plt.ylabel("Features")

In [ ]:
from sklearn.ensemble import RandomForestClassifier
radm_clf =RandomForestClassifier(max_depth =10,n_estimators=10)
radm_clf.fit(X_train,y_train)

In [ ]:
import numpy as np 
feature_rank = pd.DataFrame({'feature':X_train.columns,'importance':linreg.coef_})
feature_rank =feature_rank.sort_values('importance',ascending =False)
plt.figure(figsize=(8,6))
sn.barplot(y='feature',x='importance',data=feature_rank);

In [37]:
y_pred, std = gpr.predict(X_train, return_std=True)
y_pred=linreg.predict(X_train)


In [ ]:
test_pred_df = pd.DataFrame({'actual':y_train,'predicted':np.round(y_pred,2),'residuals':y_train-y_pred})
test_pred_df.sample(5)

In [ ]:
plt.scatterplot